In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import xlrd
from py2neo import Graph,Node,Relationship
from py2neo.database import ClientError
import re

In [1]:
# option_entities_file = 'entities.csv'
# option_relations_file = 'relations.txt'
# option_properties_file = 'properties.txt'
option_deleteAllFlag = 1
option_demoFlag = 1
option_read_csv_row =  -1 # -1 for all rows, positive number k for first k rows.
option_userName = 'neo4j'
option_passWord = '123123'
option_config_file = r''
option_entities_dir = ''
option_column_prefix = "" 
option_label = ""
option_neo4j_ip = "localhost"
option_neo4j_port = "7474"
option_pas = False
option_intesrv = True
option_decoration = False
option_remote = False

if option_pas == True:
#     option_config_file = r'E:/天远/知识图谱/天远知识图谱文档/数据血缘/PAS/PAS_config.xlsx'
    option_config_file = r'./config/PAS_config.xlsx'
    option_entities_dir = './data/pas/'
    option_column_prefix = "" 
    option_label = "数据起源"

if option_intesrv == True:
    option_config_file = r'E:/天远/知识图谱/天远知识图谱文档/天远百度/config/config_intesrv/config_v2.2.0.xlsx'
#     option_config_file = './data/config/config_v2.2.0.xlsx'
    option_entities_dir = './data/intesrv_real/'
    option_column_prefix = "plt_" 
    option_label = "智能服务"
    
if option_decoration == True:
    option_config_file = r'E:/天远/知识图谱/天远知识图谱文档/天远百度/config/config_manual/config_manual_v1.2.1.xlsx'
    option_config_file = './data/decoration/config_manual_v1.3.0.xlsx'
    option_entities_dir = './data/decoration/'
    option_column_prefix = "" 
    option_label = "装修手册"
    
if option_remote == True:
    option_passWord = '123456'
    option_neo4j_ip = "101.6.15.212"


##### debug

## 读取配置表

In [2]:
class Config:
    MAIN_FOREIGN_SHEET_NAME = '主外键对应表'
    CONCEPT_SHEET_NAME = '模式图的概念与数据库对应表'
    RELATION_SHEET_NAME = '关系表'

    DICT_CONCEPT = {"模式图中概念":"concept_ch",
                   "显示名":"display_name",
                   "数据库表/属性":"table_and_column",
                   "类型":"type",
                   "逻辑代码连接":"code",
                   "内部属性":"inner_properties_ch",
                   "内部属性对应数据库属性":"properties_col_str",
                   "逻辑代码所需数据库表":"compound_table_str",
                   "所属域":"domain"}
    DICT_CONCEPT_REVERSE = {value:key for key,value in DICT_CONCEPT.items()}
    DICT_RELATION = {"关系主语":"sub",
                   "关系谓语":"predicate",
                   "关系宾语":"obj",
                   "关系类型":"type",
                   "关系所在数据库表":"table",
                    "关系属性":"prop_ch_str",
                    "关系属性对应数据库属性":"prop_col_str"}
    DICT_RELATION_REVERSE = {value:key for key,value in DICT_RELATION.items()}
    def __init__(self, file_path, debug=False):
        self.__initVariable(file_path, debug)
    
    def __initVariable(self, file_path, debug):
        self.__file_path = file_path
        self.__mainForeign = None
        self.__concept = None
        self.__relation = None
        self.__tables_name = []
        self.__debug = debug

    def readAllContent(self):
        self.__readConcept()
        self.__readRelation()
        self.__readMainForeign()
        self.__getAllTablesInTopoSort()    
        
    def __split_compound_table(paragraph):
        results = []
        for sentence in re.split(",|，", paragraph):        
            compound = {}
            compound['main'] = sentence.split('-')[0]
            compound['foreign'] = sentence.split('-')[1]
            compound['main_table'] = compound['main'].split('.')[0]
            compound['main_col'] = option_column_prefix + compound['main'].split('.')[1]
            compound['foreign_table'] = compound['foreign'].split('.')[0]
            compound['foreign_col'] = option_column_prefix + compound['foreign'].split('.')[1]
            results.append(compound)
        return results   
    
    def __readConcept(self):
        df = pd.read_excel(open(self.__file_path, 'rb'),sheet_name=self.CONCEPT_SHEET_NAME)
        for col in df.columns:
            df[col] = df[col].astype(str).str.lower()
            df[col] = df[col].str.replace("\n","")
        df = df.rename(index=str, columns=self.DICT_CONCEPT)
        # print(df)
        df['table'] = None
        df['col'] = None
        # df['test'] = df['concept_ch'].apply(split_compound_table)
        # print(df)
        # df[df['type'] == '直接']['table'] = df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[0])
        df.loc[df['type'] == '直接','table'] = df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[0])
        df.loc[df['type'] == '直接','col'] = option_column_prefix + df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[1])
        df.loc[df['type'] == '直接','properties_col'] = df[df['type'] == '直接']['properties_col_str'].astype(str).apply(lambda x: [option_column_prefix+y for y in re.split(",|，", x) if x != "nan"] )
        df.loc[df['type'] == '直接','properties_col_ch'] = df[df['type'] == '直接']['inner_properties_ch'].astype(str).apply(lambda x: [y for y in re.split(",|，", x) if x != "nan"] )
        df.loc[(df['type'] == '合成概念'), 'table'] = df[(df['type'] == '合成概念')]['table_and_column'].astype(str)
        df.loc[(df['type'] == '合成概念') & (df['compound_table_str'] != 'nan'), 'compound_table'] = df[(df['type'] == '合成概念') & (df['compound_table_str'] != 'nan')]['compound_table_str'].astype(str).apply(Config.__split_compound_table)
        self.__concept = df
        
        
    
    def __readRelation(self):
        df = pd.read_excel(open(self.__file_path, 'rb'),sheet_name=self.RELATION_SHEET_NAME)
        for col in df.columns:
            df[col] = df[col].astype(str).str.lower()
            df[col] = df[col].str.replace("\n","")
        df = df.rename(index=str, columns=self.DICT_RELATION)
        df['prop_ch'] = df['prop_ch_str'].astype(str).apply(lambda x: [y for y in re.split(",|，", x) if x != "nan"] )
        df['prop_col'] = df['prop_col_str'].astype(str).apply(lambda x: [option_column_prefix+y for y in re.split(",|，", x) if x != "nan"] )
        self.__relation = df
        
    def __readMainForeign(self):
        df = pd.read_excel(open(self.__file_path, 'rb'),sheet_name=self.MAIN_FOREIGN_SHEET_NAME)
        for col in df.columns:
            df[col] = df[col].astype(str).str.lower()
        df['primary_table'] = df['主键'].apply(lambda x: x.split('.')[0])
        df['primary_key'] = option_column_prefix + df['主键'].apply(lambda x: x.split('.')[1])
        df['foreign_table'] = df['外键'].apply(lambda x: x.split('.')[0])
        df['foreign_key'] = option_column_prefix + df['外键'].apply(lambda x: x.split('.')[1])
        if self.__debug:
            print(df)
        self.__mainForeign = df
        
    def __getAllTablesInTopoSort(self):
        self.__toposortTablesFromMainForeign()
        self.__addTablesFromConcept()
    
    def __toposortTablesFromMainForeign(self):
        tables_name = []
        df_foreign_tobedeleted = self.__mainForeign.copy()
        count = 0
        while len(df_foreign_tobedeleted) > 0:
            if self.__debug:
                print(count)
            primary_unique = df_foreign_tobedeleted['primary_table'].unique()
            foreign_uninue = df_foreign_tobedeleted['foreign_table'].unique()
            all_table = np.concatenate((primary_unique,foreign_uninue))
            tables_name_unordered = set(all_table)
            if self.__debug:
                print(tables_name_unordered)
            foreign_table = set(df_foreign_tobedeleted['foreign_table'])
            table = tables_name_unordered.difference(foreign_table).pop()
        #     print(tables_name_unordered.difference(foreign_table))
            tables_name.append(table)
        #     print(table)
            index_tobedeleted = df_foreign_tobedeleted.loc[df_foreign_tobedeleted['primary_table'] == table].index
        #     print(index_tobedeleted)
            df_foreign_tobedeleted.drop(index_tobedeleted, inplace=True)
        #     print(df_foreign_tobedeleted[['primary_table','foreign_table']])
            count += 1   
        if self.__debug:
            print(tables_name)

        # 只依赖其他表而没有被其他表依赖的表，并没有在上一步骤中被添加进来，所以在此处添加他们
        df_foreign_tobedeleted = self.__mainForeign.copy()
        primary_unique = df_foreign_tobedeleted['primary_table'].unique()
        foreign_uninue = df_foreign_tobedeleted['foreign_table'].unique()
        all_table = np.concatenate((primary_unique,foreign_uninue))
        tables_name_unordered = set(all_table)
        table_not_added = tables_name_unordered.difference(set(tables_name))
        while len(table_not_added) > 0:
            tables_name.append(table_not_added.pop())
        if self.__debug:
            print(tables_name)
        self.__tables_name += tables_name
    
    def __addTablesFromConcept(self):
        tables_name_from_concept = set(self.__concept['table'].dropna().unique())
        tables_name_now = set(self.__tables_name)
        tables_name_to_add = tables_name_from_concept.difference(tables_name_now)
        tables_name_to_add = list(tables_name_to_add)
        self.__tables_name += tables_name_to_add
        if self.__debug:
            print("debug:self.__tables_name=",self.__tables_name)
    
    def getConcept(self):
        return self.__concept
    
    def getRelation(self):
        return self.__relation
    
    def getMainForeign(self):
        return self.__mainForeign
    
    def getTables(self):
        return self.__tables_name
    
    def getConceptOfOneTable(self, table_name):
        concept = self.getConcept()
        return concept[concept['table'] == table_name].reset_index()
    
    def getRelationOfOneTable(self, table_name):
        relation = self.getRelation()
        return relation[relation['table'] == table_name].reset_index()

In [5]:
config = Config(option_config_file, debug=False)
config.readAllContent()

In [6]:
config.getRelationOfOneTable('plt_intesrv_tycustomer')

,index,id,sub,predicate,obj,table,type,prop_ch_str,prop_col_str,prop_ch,prop_col
0,3,4,客户,属于,客户类型,plt_intesrv_tycustomer,nan,nan,nan,[],[]
1,4,5,客户,属于,客户等级,plt_intesrv_tycustomer,nan,nan,nan,[],[]


In [7]:
config.getConceptOfOneTable('plt_intesrv_tycustomer')

,index,concept_ch,display_name,table_and_column,type,inner_properties_ch,properties_col_str,code,compound_table_str,domain,table,col,properties_col,properties_col_ch,compound_table
0,0,客户,客户名称,plt_intesrv_tycustomer.oid,直接,"客户号,客户名称,工地位置,办公地位置,住所位置","id,customername，mainlonlat,customeraddress,hom...",nan,nan,智能服务,plt_intesrv_tycustomer,plt_oid,"[plt_id, plt_customername, plt_mainlonlat, plt...","[客户号, 客户名称, 工地位置, 办公地位置, 住所位置]",NaN
1,1,客户类型,unique_id,plt_intesrv_tycustomer.customertype,直接,nan,nan,nan,nan,智能服务,plt_intesrv_tycustomer,plt_customertype,[],[],NaN
2,2,客户等级,unique_id,plt_intesrv_tycustomer.custlevel,直接,nan,nan,nan,nan,智能服务,plt_intesrv_tycustomer,plt_custlevel,[],[],NaN


In [8]:
config.getConcept()

,concept_ch,display_name,table_and_column,type,inner_properties_ch,properties_col_str,code,compound_table_str,domain,table,col,properties_col,properties_col_ch,compound_table
0,客户,客户名称,plt_intesrv_tycustomer.oid,直接,"客户号,客户名称,工地位置,办公地位置,住所位置","id,customername，mainlonlat,customeraddress,hom...",nan,nan,智能服务,plt_intesrv_tycustomer,plt_oid,"[plt_id, plt_customername, plt_mainlonlat, plt...","[客户号, 客户名称, 工地位置, 办公地位置, 住所位置]",NaN
1,客户类型,unique_id,plt_intesrv_tycustomer.customertype,直接,nan,nan,nan,nan,智能服务,plt_intesrv_tycustomer,plt_customertype,[],[],NaN
2,客户等级,unique_id,plt_intesrv_tycustomer.custlevel,直接,nan,nan,nan,nan,智能服务,plt_intesrv_tycustomer,plt_custlevel,[],[],NaN
3,联系人,联系人姓名,plt_intesrv_tylinkman.oid,直接,"联系人号，联系人姓名,联系人电话，角色","id,linkmanname,linkmanphone,linkmantype",nan,nan,智能服务,plt_intesrv_tylinkman,plt_oid,"[plt_id, plt_linkmanname, plt_linkmanphone, pl...","[联系人号, 联系人姓名, 联系人电话, 角色]",NaN
4,工单,工单号,plt_intesrv_tywork.oid,直接,"工单号,名称，是否停机,工单注册时间，挖机的工作小时数，故障发生时间，工作开始时间，工作结束时间","id,workname，isshutdown,createtime,workhours，fa...",nan,nan,智能服务,plt_intesrv_tywork,plt_oid,"[plt_id, plt_workname, plt_isshutdown, plt_cre...","[工单号, 名称, 是否停机, 工单注册时间, 挖机的工作小时数, 故障发生时间, 工作开始...",NaN
5,工单处理时长,nan,plt_intesrv_tywork,合成概念,nan,nan,test1,nan,智能服务,plt_intesrv_tywork,None,NaN,NaN,NaN
6,当前八级报警级别,级别,nan,合成,级别,nan,nan,nan,智能服务,None,None,NaN,NaN,NaN
7,历史八级报警级别,nan,nan,合成,级别,nan,nan,nan,智能服务,None,None,NaN,NaN,NaN
8,当前工单状态,unique_id,plt_intesrv_tywork.workstatus,直接,nan,nan,nan,nan,智能服务,plt_intesrv_tywork,plt_workstatus,[],[],NaN
9,历史工单状态,nan,nan,合成,状态,nan,nan,nan,智能服务,None,None,NaN,NaN,NaN


In [9]:
config.getMainForeign()

,主键,外键,primary_table,primary_key,foreign_table,foreign_key
0,plt_intesrv_tywork.oid,plt_intesrv_tyworkdispatch.tyworkoid,plt_intesrv_tywork,plt_oid,plt_intesrv_tyworkdispatch,plt_tyworkoid
1,plt_intesrv_tywork.oid,plt_intesrv_tyjob.tyworkoid,plt_intesrv_tywork,plt_oid,plt_intesrv_tyjob,plt_tyworkoid
2,plt_intesrv_tywork.oid,plt_intesrv_typartdeliveryinfo.tyworkoid,plt_intesrv_tywork,plt_oid,plt_intesrv_typartdeliveryinfo,plt_tyworkoid
3,plt_intesrv_typartdeliveryinfo.oid,plt_intesrv_typartdelivery.typartdeliveryinfooid,plt_intesrv_typartdeliveryinfo,plt_oid,plt_intesrv_typartdelivery,plt_typartdeliveryinfooid
4,plt_intesrv_tycustomer.oid,plt_intesrv_tydevice.tycustomeroid,plt_intesrv_tycustomer,plt_oid,plt_intesrv_tydevice,plt_tycustomeroid
5,plt_intesrv_tycustomer.oid,plt_intesrv_tylinkman.customeroid,plt_intesrv_tycustomer,plt_oid,plt_intesrv_tylinkman,plt_customeroid
6,plt_intesrv_tyworkdispatch.oid,plt_intesrv_tyworkmileage.tyworkdispatchoid,plt_intesrv_tyworkdispatch,plt_oid,plt_intesrv_tyworkmileage,plt_tyworkdispatchoid
7,plt_intesrv_tydevice.oid,plt_intesrv_tywork.tydeviceoid,plt_intesrv_tydevice,plt_oid,plt_intesrv_tywork,plt_tydeviceoid
8,plt_org_user.oid,plt_intesrv_typartdeliveryinfo.stockupoid,plt_org_user,plt_oid,plt_intesrv_typartdeliveryinfo,plt_stockupoid
9,plt_org_user.oid,plt_intesrv_tywork.creator,plt_org_user,plt_oid,plt_intesrv_tywork,plt_creator


In [10]:
config.getRelation()

,id,sub,predicate,obj,table,type,prop_ch_str,prop_col_str,prop_ch,prop_col
0,1,联系人,属于,人,nan,nan,nan,nan,[],[]
1,2,联系人,具有,联系优先级,nan,nan,nan,nan,[],[]
2,3,客户,具有,联系人,plt_intesrv_tylinkman,nan,nan,nan,[],[]
3,4,客户,属于,客户类型,plt_intesrv_tycustomer,nan,nan,nan,[],[]
4,5,客户,属于,客户等级,plt_intesrv_tycustomer,nan,nan,nan,[],[]
5,6,客户,使用,发票模板,nan,nan,nan,nan,[],[]
6,7,客户,具有,信誉信息,nan,nan,nan,nan,[],[]
7,8,客户,具有,客户满意度,nan,nan,nan,nan,[],[]
8,9,挖掘机,属于,吨级,plt_intesrv_tydevice,nan,nan,nan,[],[]
9,10,挖掘机,属于,客户,plt_intesrv_tydevice,nan,nan,nan,[],[]


In [11]:
config.getTables()

['plt_intesrv_tysrvperson',
 'plt_org_user',
 'plt_intesrv_tycustomer',
 'plt_intesrv_tydevice',
 'plt_intesrv_tywork',
 'plt_intesrv_typartdeliveryinfo',
 'plt_intesrv_tyworkdispatch',
 'plt_intesrv_typartdelivery',
 'plt_intesrv_tyjob',
 'plt_intesrv_tyworkmileage',
 'plt_intesrv_tylinkman',
 'plt_intesrv_tytodorepair']

#### debug

## 读取实体数据

In [12]:
class EntityInRdb:
    def __init__(self, entity_dir, tables_name, row_use = -1):
        self.__tables = {}
        self.__tables_name = tables_name
        self.__entity_dir = entity_dir
        self.__row_use = row_use
        self.__table_row_total_length = 0
        
    def readCsv(self, print_flag = True):
        tables_name = self.__tables_name
        for table_name in tables_name:
            if print_flag:
                print("start to access table " + table_name)
            path = self.__entity_dir + table_name + ".csv"
            df = pd.read_csv(path,sep=';',encoding='utf-16')
            for key in list(df.columns.values):  # 如果不这样的话，可能会读入int64的数字，但是neo4j不支持int64
                df[key] = df[key].astype("object")
            if print_flag:
                print(df.shape)
            self.__table_row_total_length += df.shape[0]
            row_use = df.shape[0] if self.__row_use == -1 else self.__row_use
            self.__tables[table_name] = df[:row_use]
            if print_flag:
                print("end to access table " + table_name)
        if print_flag:
            print("all tables have %d rows. We use %s rows in each table."
                  %(self.__table_row_total_length,"all" if self.__row_use == -1 else str(self.__row_use)))
            
    def getAllTable(self):
        return self.__tables
    
    def getAllTableName(self):
        return self.__tables_name
   
    def getEntityDir(self):
        return self.__entity_dir
    
    def getRowUse(self):
        return self.__row_use
    
    def getTableRowTotalLength(self):
        return self.__table_row_total_length
        
    def getOneTable(self,table_name):
        if table_name not in self.getAllTableName():
            raise Exception("table_name %s doesn't exist"%(table_name))
        return self.getAllTable()[table_name]

In [13]:
entityInRdb = EntityInRdb(option_entities_dir, config.getTables(), option_read_csv_row)

In [14]:
entityInRdb.readCsv()

start to access table plt_intesrv_tysrvperson
(15, 17)
end to access table plt_intesrv_tysrvperson
start to access table plt_org_user
(43, 14)
end to access table plt_org_user
start to access table plt_intesrv_tycustomer
(9035, 34)
end to access table plt_intesrv_tycustomer
start to access table plt_intesrv_tydevice
(13098, 53)
end to access table plt_intesrv_tydevice
start to access table plt_intesrv_tywork
(444, 86)
end to access table plt_intesrv_tywork
start to access table plt_intesrv_typartdeliveryinfo
(340, 46)
end to access table plt_intesrv_typartdeliveryinfo
start to access table plt_intesrv_tyworkdispatch
(271, 27)
end to access table plt_intesrv_tyworkdispatch
start to access table plt_intesrv_typartdelivery
(4029, 41)
end to access table plt_intesrv_typartdelivery
start to access table plt_intesrv_tyjob
(7155, 26)
end to access table plt_intesrv_tyjob
start to access table plt_intesrv_tyworkmileage
(282, 21)
end to access table plt_intesrv_tyworkmileage
start to access tab

In [15]:
entityInRdb.getOneTable('plt_intesrv_tyjob')

,plt_oid,plt_owner,plt_createtime,plt_creator,plt_currentprocess,plt_id,plt_lastmodifytime,plt_lastmodifier,plt_tyworkoid,plt_jobname,...,plt_sequence,plt_tyexpertworkoid,plt_jobsummary,plt_partoids,plt_tyexpertjoboid,plt_result,plt_desc,plt_imgurls,plt_autocheckresult,plt_manualcheckresult
0,86EB5026C53C5B49BACC22B0060C92A2,NaN,2019-02-18 09:41:10.000000,9C92E891E9AE534DB685737DE467A9D0,NaN,TJ20190218094110262,2019-02-18 09:41:10.000000,ECB1037417BAF14183EF0FB89043A4C2,6874D6DAF4BD3A43AF659B0D3EF1DE0C,清洗和检查散热器片、油冷却器片、中冷器片、燃油冷却器片和空调冷凝器片（仅限装有空调的 机器）,...,5,D3A4244539A79D47BC282DA38CE7FAD4,NaN,NaN,NaN,NaN,NaN,https://dummyimage.com/600x400/666/ddd.jpg&tex...,NaN,NaN
1,59954BF28A51794EA4B14D667B8DD876,NaN,2019-02-18 09:41:10.000000,9C92E891E9AE534DB685737DE467A9D0,NaN,TJ20190218094110933,2019-02-18 09:41:10.000000,ECB1037417BAF14183EF0FB89043A4C2,6874D6DAF4BD3A43AF659B0D3EF1DE0C,清洁空调的换气/循环过滤器,...,6,D3A4244539A79D47BC282DA38CE7FAD4,NaN,NaN,NaN,NaN,NaN,https://dummyimage.com/600x400/666/ddd.jpg&tex...,NaN,NaN
2,F321C59F45F85B4E8C66057269AF0E55,NaN,2019-02-18 09:41:10.000000,9C92E891E9AE534DB685737DE467A9D0,NaN,TJ20190218094110352,2019-02-18 09:41:10.000000,ECB1037417BAF14183EF0FB89043A4C2,6874D6DAF4BD3A43AF659B0D3EF1DE0C,更换液压油箱内通气装置滤芯,...,7,D3A4244539A79D47BC282DA38CE7FAD4,NaN,NaN,NaN,NaN,NaN,https://dummyimage.com/600x400/666/ddd.jpg&tex...,NaN,NaN
3,7C95B07FBF38AC459568DB5155F12C71,NaN,2019-02-18 09:41:10.000000,9C92E891E9AE534DB685737DE467A9D0,NaN,TJ20190218094110716,2019-02-18 09:41:10.000000,ECB1037417BAF14183EF0FB89043A4C2,6874D6DAF4BD3A43AF659B0D3EF1DE0C,更换燃油箱附加通气装置滤芯（仅装有大容量燃油预过滤器的机器）,...,8,D3A4244539A79D47BC282DA38CE7FAD4,NaN,NaN,NaN,NaN,NaN,https://dummyimage.com/600x400/666/ddd.jpg&tex...,NaN,NaN
4,B11346DFAE70E847B17E1549CB24F6F6,NaN,2019-02-18 09:41:10.000000,9C92E891E9AE534DB685737DE467A9D0,NaN,TJ20190218094110087,2019-02-18 09:41:10.000000,ECB1037417BAF14183EF0FB89043A4C2,6874D6DAF4BD3A43AF659B0D3EF1DE0C,检查回转机构箱内的油位，加油,...,9,D3A4244539A79D47BC282DA38CE7FAD4,NaN,NaN,NaN,NaN,NaN,https://dummyimage.com/600x400/666/ddd.jpg&tex...,NaN,NaN


## 导入Neo4j

应该对neo4jconnection进行权限管理，删除、写入、查询三个操作的权限应依次减小，在到处传neo4jconnection的时候，应限制有些class只能用它进行查询操作，某些class才可以进行写操作。
要能从大权限变成小权限，但是反之不可以。

In [17]:
class Neo4jSearch:
    pass

class Neo4jSearchCreate:
    pass

In [18]:
class Neo4jConnection:
    def __init__(self, ip, port, userName, password):
        self.__ip = ip
        self.__port = port
        self.__userName = userName
        self.__password = password
        self.__graph = None
        
    def connect(self):
        self.__graph = Graph('http://%s:%s'%(self.__ip,self.__port),username=self.__userName,password=self.__password)
       
    def getGraph(self):
        return self.__graph
    
    def deleteAllOrByLabel(self, labelToDetele=None):
        if labelToDetele is None:
            self.__graph.delete_all()
        else:
            self.__graph.run("MATCH (n:%s) OPTIONAL MATCH (n)-[r]-() DELETE r,n"%(labelToDetele)) 
    
    def createNode(self, node):
        self.__graph.create(node)
    
    def createRelation(self, relation):
        self.__graph.create(relation)
        
    def createGraph(self, graph):
        self.__graph.create(graph)
    
    def searchNodeByLableAndUniqueid(self, label, uniqueid):
        return self.__graph.nodes.match(label, uniqueid=uniqueid).first()

In [19]:
neo = Neo4jConnection(option_neo4j_ip,option_neo4j_port,option_userName,option_passWord)
neo.connect()

In [20]:
neo.deleteAllOrByLabel('相关信息')

In [21]:
neo.deleteAllOrByLabel()

### 模式图

In [22]:
concept = config.getConcept()
concept

,concept_ch,display_name,table_and_column,type,inner_properties_ch,properties_col_str,code,compound_table_str,domain,table,col,properties_col,properties_col_ch,compound_table
0,客户,客户名称,plt_intesrv_tycustomer.oid,直接,"客户号,客户名称,工地位置,办公地位置,住所位置","id,customername，mainlonlat,customeraddress,hom...",nan,nan,智能服务,plt_intesrv_tycustomer,plt_oid,"[plt_id, plt_customername, plt_mainlonlat, plt...","[客户号, 客户名称, 工地位置, 办公地位置, 住所位置]",NaN
1,客户类型,unique_id,plt_intesrv_tycustomer.customertype,直接,nan,nan,nan,nan,智能服务,plt_intesrv_tycustomer,plt_customertype,[],[],NaN
2,客户等级,unique_id,plt_intesrv_tycustomer.custlevel,直接,nan,nan,nan,nan,智能服务,plt_intesrv_tycustomer,plt_custlevel,[],[],NaN
3,联系人,联系人姓名,plt_intesrv_tylinkman.oid,直接,"联系人号，联系人姓名,联系人电话，角色","id,linkmanname,linkmanphone,linkmantype",nan,nan,智能服务,plt_intesrv_tylinkman,plt_oid,"[plt_id, plt_linkmanname, plt_linkmanphone, pl...","[联系人号, 联系人姓名, 联系人电话, 角色]",NaN
4,工单,工单号,plt_intesrv_tywork.oid,直接,"工单号,名称，是否停机,工单注册时间，挖机的工作小时数，故障发生时间，工作开始时间，工作结束时间","id,workname，isshutdown,createtime,workhours，fa...",nan,nan,智能服务,plt_intesrv_tywork,plt_oid,"[plt_id, plt_workname, plt_isshutdown, plt_cre...","[工单号, 名称, 是否停机, 工单注册时间, 挖机的工作小时数, 故障发生时间, 工作开始...",NaN
5,工单处理时长,nan,plt_intesrv_tywork,合成概念,nan,nan,test1,nan,智能服务,plt_intesrv_tywork,None,NaN,NaN,NaN
6,当前八级报警级别,级别,nan,合成,级别,nan,nan,nan,智能服务,None,None,NaN,NaN,NaN
7,历史八级报警级别,nan,nan,合成,级别,nan,nan,nan,智能服务,None,None,NaN,NaN,NaN
8,当前工单状态,unique_id,plt_intesrv_tywork.workstatus,直接,nan,nan,nan,nan,智能服务,plt_intesrv_tywork,plt_workstatus,[],[],NaN
9,历史工单状态,nan,nan,合成,状态,nan,nan,nan,智能服务,None,None,NaN,NaN,NaN


In [23]:
def notNan(value):
    return value == value

def isNan(value):
    return value != value

def change2float(x):
    if re.match("^\d+?(\.\d+?)?$", str(x)) is None:
        return x
    else:
        return float(x)
# change2float('1.09')

In [24]:
class SubGraph:
    def __init__(self, neo4jConnection):
        self.__nodes = {}
        self.__relations = []
        self.__neo4jConnection = neo4jConnection
    
    def clearGraph(self):
        self.__nodes.clear()
        self.__relations.clear()
        
    def addNode(self, key, aNode):
        self.__nodes[key] = aNode
        
    def existKeyInNodeList(self, key):
        return key in self.__nodes.keys()
    
    def getOneNode(self, key):
        return self.__nodes[key]
        
    def addRelation(self, aRelation):
        self.__relations.append(aRelation)
    
    def getAllNode(self):
        return self.__nodes.values()
    
    def getAllRelation(self):
        return self.__relations
    
    def createGraphInNeo4jAndClear(self):
        self.createGraphInNeo4j()
        self.clearGraph()
        
    def createGraphInNeo4j(self):
        for aNode in self.getAllNode():
            self.__neo4jConnection.createGraph(aNode) 
        for aRelation in self.getAllRelation():
            self.__neo4jConnection.createGraph(aRelation)

In [25]:
class ConceptGraph:
    CONCEPT_LABEL = '概念节点'
    CONCEPT_CHINESE_COLUMN_NAME = 'concept_ch'
    DOMAIN_COLUMN_NAME = 'domain'
    SUBJECT_COLUMN_NAME = 'sub'
    OBJECT_COLUMN_NAME = 'obj'
    RELATION_NAME_COLUMN_NAME = 'predicate'
    RELATION_PROPERTY_NAME_NEO4J = 'id_relation'
    RELATION_PROPERTY_NAME_RDB = 'id'
    def __init__(self, config, neo4jConnection):
        self.__config = config
        self.__neo4jConnection = neo4jConnection
        self.__subgraph = SubGraph(neo4jConnection)
    
    def constructGraph(self):
        self.constructAllNodes()
        self.constructAllRelations()
        self.createGraphInNeo4j()
        
    def constructAllNodes(self):
        conceptConfig = self.__config.getConcept()
#         self.__nodes = {} # 每次create之后都会清空
        for index in tqdm(list(conceptConfig.index)):
#             index = int(index)
            aConfig = conceptConfig.loc[index].to_dict()
            conceptNameCh = aConfig[self.CONCEPT_CHINESE_COLUMN_NAME]
            aNode = self.constructOneNode(aConfig)
            self.__subgraph.addNode(conceptNameCh, aNode)
            
    def constructOneNode(self,theConfig):
        aNode = Node()
        self.addLabel2NodeInplace(aNode, theConfig)
        self.addProperties2NodeInplace(aNode, theConfig)
        return aNode
    
    def addLabel2NodeInplace(self,aNode,theConfig):
        aNode.add_label(theConfig[self.CONCEPT_CHINESE_COLUMN_NAME])
        aNode.add_label(theConfig[self.DOMAIN_COLUMN_NAME])
        aNode.add_label(self.CONCEPT_LABEL)
    
    def addProperties2NodeInplace(self, aNode, theConfig):
        for col_name in list(theConfig.keys()):
            value = theConfig[col_name]
            if notNan(value):
                value = str(value)
            else:
                value = ''
            if col_name in self.__config.DICT_CONCEPT_REVERSE.keys():
                col_name_ch = self.__config.DICT_CONCEPT_REVERSE[col_name]
            else:
                col_name_ch = col_name
            aNode[col_name_ch] = value
        
    def constructAllRelations(self):
        relationConfig = self.__config.getRelation()
        for index in tqdm(list(relationConfig.index)):
            aRelationConfig = relationConfig.loc[index].to_dict()
            aRelation = self.constructOneRelation(aRelationConfig)
            self.__subgraph.addRelation(aRelation)
        
    def constructOneRelation(self, theRelationConfig):
        subName = theRelationConfig[self.SUBJECT_COLUMN_NAME]
        objName = theRelationConfig[self.OBJECT_COLUMN_NAME]
        relationName = theRelationConfig[self.RELATION_NAME_COLUMN_NAME]
        aRelation = Relationship(self.__subgraph.getOneNode(subName), relationName, self.__subgraph.getOneNode(objName))
        self.addProperties2RelationInplace(aRelation, theRelationConfig)
        return aRelation
    
    def addProperties2RelationInplace(self, theRelation, theRelationConfig):
        theRelation[self.RELATION_PROPERTY_NAME_NEO4J] = theRelationConfig[self.RELATION_PROPERTY_NAME_RDB]
        for col_name in list(theRelationConfig.keys()):
            value = theRelationConfig[col_name]
            if notNan(value):
                value = str(value)
            else:
                value = ''
            if col_name in self.__config.DICT_RELATION_REVERSE.keys():
                col_name_ch = self.__config.DICT_RELATION_REVERSE[col_name]
            else:
                col_name_ch = col_name
            theRelationConfig[col_name_ch] = value
    
    def createGraphInNeo4j(self):
        self.__subgraph.createGraphInNeo4jAndClear()

In [26]:
# 输入一行数据以及一个实体概念的config，能返回该概念对应的Node
class InstanceNodeDirect:
    INSTANCE_LABEL = '实例节点'
    CONCEPT_CHINESE_COLUMN_NAME = 'concept_ch'
    UNIQUE_ID_COLUMN_NAME = 'col'
    DOMAIN_COLUMN_NAME = 'domain'
    PROPERTIES_COL_COLUMN_NAME = 'properties_col'
    PROPERTIES_COL_CH_COLUMN_NAME = 'properties_col_ch'
    UNIQUE_ID_PROPERTITY = 'unique_id'
    TYPE_PROPERTITY = 'type'
    
    def __init__(self, entityInRdbRow, conceptConfigOneNode):
        self.__entityInRdbRow = entityInRdbRow
        self.__conceptConfigOneNode = conceptConfigOneNode
        self.__node = Node()
        
    def getSource(self):
        return self.__entityInRdbRow
    
    def getSourceByKey(self, key):
        return self.getSource()[key]
    
    def getConfig(self):
        return self.__conceptConfigOneNode
    
    def getConfigByKey(self, key):
        return self.getConfig()[key]
    
    def getNode(self):
        return self.__node
        
    def initNode(self):
        self.__node = Node()
        
    def addLabel2Node(self, label):
        self.getNode().add_label(label)
    
    def addProperty2Node(self, key, value):
        self.getNode()[key] = value
    
    def constructNode(self):
        self.initNode()
        self.addBasicInfo()
        self.addAllLabels()
        self.addUserSpecifiedProperties()
        self.addDisplayNameProperty()
        
    def addBasicInfo(self):
        unique_id_col_name = self.getConfigByKey(self.UNIQUE_ID_COLUMN_NAME)
        unique_id = self.getSourceByKey(unique_id_col_name)
        concept_ch = self.getConfigByKey(self.CONCEPT_CHINESE_COLUMN_NAME)
        self.addProperty2Node(self.UNIQUE_ID_PROPERTITY, unique_id)
        self.addProperty2Node(self.TYPE_PROPERTITY, concept_ch)
        
    def addAllLabels(self):
        concept_ch = self.getConfigByKey(self.CONCEPT_CHINESE_COLUMN_NAME)
        self.addLabel2Node(concept_ch)
        domain = self.getConfigByKey(self.DOMAIN_COLUMN_NAME)
        self.addLabel2Node(domain)
        self.addLabel2Node(self.INSTANCE_LABEL)
        
    def addUserSpecifiedProperties(self):
        prop_key_list = self.getConfigByKey(self.PROPERTIES_COL_COLUMN_NAME)
        prop_key_ch_list =  self.getConfigByKey(self.PROPERTIES_COL_CH_COLUMN_NAME)
        prop_num = len(prop_key_list)
        prop_ch_num = len(prop_key_ch_list)
        assert prop_num == prop_ch_num ,"properties_col=%s, properties_col_ch=%s. They have different length."%(prop_num,prop_ch_num)
        
        for i in range(prop_num):
            prop_key = prop_key_list[i]
            prop_key_ch = prop_key_ch_list[i]
            prop_value = self.getSourceByKey(prop_key)
            if notNan(prop_value):
                prop_value = change2float(prop_value)
            else:
                prop_value = ''
            self.addProperty2Node(prop_key_ch, prop_value)
            
    def addDisplayNameProperty(self):
        # not implemented
        pass

In [27]:
# 输入一行数据以及一个实体概念的config，能返回该概念对应的Node
class InstanceNodeConstructor:
    TYPE_COLUMN_NAME = 'type'
    TYPE_DIRECT = '直接'
    TYPE_COMPOUND = '合成概念'
    CONCEPT_CHINESE_COLUMN_NAME = 'concept_ch'
    UNIQUE_ID_COLUMN_NAME = 'col'
    
    def __init__(self, entityInRdbRow, conceptConfigOneNode, neo4jConnection):
        self.__entityInRdbRow = entityInRdbRow
        self.__conceptConfigOneNode = conceptConfigOneNode
        self.__neo4jConnection = neo4jConnection
        self.__node = None
        
    def __getEntity(self):
        return self.__entityInRdbRow
    
    def __getEntityByKey(self, key):
        return self.__getEntity()[key]
    
    def __getConfig(self):
        return self.__conceptConfigOneNode
    
    def __getConfigByKey(self, key):
        return self.__getConfig()[key]
    
    def getNode(self):
        return self.__node
    
    def __setNode(self, aNode):
        self.__node = aNode
        
    def existsNode(self):
        return self.getNode() is not None
        
        
    def constructNode(self):
        conceptType = self.__getConfigByKey(self.TYPE_COLUMN_NAME)
        if conceptType == self.TYPE_DIRECT:
            aNode = self.searchOrConstructOneDirectNode()
        elif conceptType == self.TYPE_COMPOUND:
            aNode = self.searchOrConstructOneCompoundNode()
        else:
            # 暂时不做任何操作，需要的时候再做补充
            pass
        self.__setNode(aNode)
    
    def searchOrConstructOneDirectNode(self):
#         print("debug:conceptConfigOneNode=",conceptConfigOneNode)
        unique_id_col_name = self.__getConfigByKey(self.UNIQUE_ID_COLUMN_NAME)
#         print(conceptConfigOneNode)
        unique_id = self.__getEntityByKey(unique_id_col_name)
        if isNan(unique_id):
            return None
        concept_ch = self.__getConfigByKey(self.CONCEPT_CHINESE_COLUMN_NAME)
        nodeBefore = self.__neo4jConnection.searchNodeByLableAndUniqueid(concept_ch, unique_id)
        if nodeBefore is None:
            nodeNow = self.constructOneDirectNode()
        else:
            nodeNow = nodeBefore
        return nodeNow

    def constructOneDirectNode(self):
        entityOneRow = self.__getEntity()
        conceptConfigOneNode = self.__getConfig()
        aNode = InstanceNodeDirect(entityOneRow, conceptConfigOneNode)
        aNode.constructNode()
        return aNode.getNode()   
    
    def searchOrConstructOneCompoundNode(self):
        return None

In [33]:
class InstanceRelationConstructor:
    
    def __init__(self, entityInRdb, conceptConfig, theRelationConfig, 
                 mainForeignConfig, neo4jconnection, nodesConstructedInThisRow):
        self.__entityInRdb = entityInRdb
        self.__conceptConfig = conceptConfig
        self.__theRelationConfig = theRelationConfig
        self.__mainForeignConfig = mainForeignConfig
        self.__neo4jconnection = neo4jconnection
        self.__nodesConstructedInThisRow = nodesConstructedInThisRow
        self.__relation = None
    
    def getRelation(self):
        return self.__relation
    
    def setRelation(self, theRelation):
        self.__relation = theRelation
 
    def existsRelation(self):
        return self.getRelation() is not None

    def constructRelation(self):
        subNode = self.constructSubjectNode()
        objNode = self.constructObjectNode()
        self.constructRelationWithSubAndObj(subNode, objNode)
    
    def constructSubjectNode(self):
        # not implemented
        return None
    
    def constructObjectNode(self):
        # not implemented
        return None
    
    def constructRelationWithSubAndObj(self, subNode, objNode):
        if subNode is None or objNode is None:
            return
        typeName = "xxx"
        theRelation = Relationship(subNode, typeName, objNode)
        self.addPropertiesToRelationInplace(theRelation)
        # not implemented
        self.setRelation(theRelation)

    def addPropertiesToRelationInplace(self, theRelation):
        # not implemented
        return


In [34]:
# 输入一行数据以及本表的config，就能直接在Neo4j中生成该行包含的所有实体
class InstanceRow:
    TYPE_COLUMN_NAME = 'type'
    TYPE_DIRECT = '直接'
    TYPE_COMPOUND = '合成概念'
    CONCEPT_CHINESE_COLUMN_NAME = 'concept_ch'
    UNIQUE_ID_COLUMN_NAME = 'col'
    
    def __init__(self, entityInRdbnRow, conceptConfigOneTable, relationConfigOneTable, mainForeignConfig, neo4jConnection):
        self.__entityInRdbnRow = entityInRdbnRow
        self.__conceptConfigOneTable = conceptConfigOneTable
        self.__relationConfigOneTable = relationConfigOneTable
        self.__mainForeignConfig = mainForeignConfig
        self.__neo4jConnection = neo4jConnection
        self.__subgraph = SubGraph(neo4jConnection)
             
            
    def __getEntity(self):
        return self.__entityInRdbnRow
    
    def __getEntityByKey(self, key):
        return self.__entityInRdbnRow[key]
    
    def __getConceptConfig(self):
        return self.__conceptConfigOneTable
    
    def __getConceptNum(self):
        return len(self.__conceptConfigOneTable)
        
    def __getOneConceptInDictByIndex(self, indexRow):
        return self.__conceptConfigOneTable.loc[indexRow].to_dict()
    
    def __getRelationConfig(self):
        return self.__relationConfigOneTable
    
    def __getRelationNum(self):
        return len(self.__relationConfigOneTable)
               
    def __getOneRelationInDictByIndex(self, indexRow):
        return self.__relationConfigOneTable.loc[indexRow].to_dict()
    
    def __getMainForeignConfig(self):
        return self.__mainForeignConfig
    
    def constructGraphAndWriteToNeo4j(self):
        self.constructGraph()
        self.createGraphInNeo4j()
        
    def constructGraph(self):
        self.constructAllNodes()
        self.constructAllRelations()
        
    def constructAllNodes(self):
        conceptNum = self.__getConceptNum()
        for i_concept in range(conceptNum):
            conceptConfigOneNode = self.__getOneConceptInDictByIndex(i_concept)
            aNodeConstructor = InstanceNodeConstructor(self.__getEntity(), conceptConfigOneNode, self.__neo4jConnection)
            aNodeConstructor.constructNode()
            if aNodeConstructor.existsNode():
                concept_ch = conceptConfigOneNode[self.CONCEPT_CHINESE_COLUMN_NAME]
                aNode = aNodeConstructor.getNode()
                self.__subgraph.addNode(concept_ch, aNode)

    def constructAllRelations(self):
        relationNum = self.__getRelationNum()
        for i_relation in range(relationNum):
            aRelationConfig = self.__getOneRelationInDictByIndex(i_relation)
            aRelationConstructor = InstanceRelationConstructor(self.__getEntity(), self.__getConceptConfig(), 
                                                               aRelationConfig, self.__getMainForeignConfig(), 
                                                               self.__neo4jConnection, self.__subgraph.getAllNode())
            aRelationConstructor.constructRelation()
            if aRelationConstructor.existsRelation():
                aRelation = aRelationConstructor.getRelation()
                self.__subgraph.addRelation(aRelation)
                
    
    def createGraphInNeo4j(self):
        self.__subgraph.createGraphInNeo4jAndClear()

    
    def constructRelationInOneRow(self, relationConfig):
        pass

In [35]:
class InstanceGraph:
    INSTANCE_LABEL = '实例节点'
    TYPE_COLUMN_NAME = 'type'
    TYPE_DIRECT = '直接'
    TYPE_COMPOUND = '合成概念'
    CONCEPT_CHINESE_COLUMN_NAME = 'concept_ch'
    UNIQUE_ID_COLUMN_NAME = 'col'
    DOMAIN_COLUMN_NAME = 'domain'
    PROPERTIES_COL_COLUMN_NAME = 'properties_col'
    PROPERTIES_COL_CH_COLUMN_NAME = 'properties_col_ch'
    UNIQUE_ID_PROPERTITY = 'unique_id'
    TYPE_PROPERTITY = 'type'
    
    def __init__(self, config, entityInRdb, neo4jConnection):
        self.__config = config
        self.__entityInRdb = entityInRdb
        self.__neo4jConnection = neo4jConnection
        self.__subgraph = SubGraph(neo4jConnection)
    
    def constructGraph(self):
        table_name_all = self.__config.getTables()
        count = 0
        for table_name in table_name_all:
            print(table_name)
            self.constructOneTableGraph(table_name)
            count += 1
#             if count >= 10:
#                 break
    
    def constructOneTableGraph(self,table_name):
        entityInRdbOneTable = self.__entityInRdb.getOneTable(table_name)
        conceptConfigOneTable = self.__config.getConceptOfOneTable(table_name)
        relationConfigOneTable = self.__config.getRelationOfOneTable(table_name)
        mainForeignConfig = self.__config.getMainForeign()
#         print(entityInRdbOneTable)
#         print(conceptConfigOneTable)
#         print(relationConfigOneTable)
        row_num = len(entityInRdbOneTable)
        for i_row in tqdm(range(row_num)):
            entityOneRow = entityInRdbOneTable.loc[i_row].to_dict()
#             print(type(entityInRdbOneTable))
#             print(entityInRdbOneTable)
#             print(entityInRdbOneTable.index)
#             print(entityOneRow)
            self.constructOneRowGraph(entityOneRow, conceptConfigOneTable, relationConfigOneTable)
    
    def constructOneRowGraph(self, entityOneRow, conceptConfig, relationConfig):
        mainForeignConfig = self.__config.getMainForeign()
        aInstanceRow = InstanceRow(entityOneRow, conceptConfig, relationConfig, mainForeignConfig, self.__neo4jConnection)
        aInstanceRow.constructGraphAndWriteToNeo4j()
        

In [36]:
class ToolForTransferDataToNeo4j:

    def __init__(self, config, entityInRdb, neo4jConnection):
        self.__config = config
        self.__entityInRdb = entityInRdb
        self.__neo4jConnection = neo4jConnection
        self.__conceptGraph = ConceptGraph(config, neo4jConnection)
        self.__instanceGraph = InstanceGraph(config, entityInRdb, neo4jConnection)
        self.__conceptGraphInNeo4j = None
        
        self.__nodesDict = {}
        self.__relationsList = []
        self.__subgraph = None
    
    def constructAllEntitiesAndRelations(self):
#         self.__conceptGraph.constructGraph()
        self.__instanceGraph.constructGraph()
        

In [37]:
neo.deleteAllOrByLabel()

transfer = ToolForTransferDataToNeo4j(config, entityInRdb, neo)

transfer.constructAllEntitiesAndRelations()

plt_intesrv_tysrvperson


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 13.15it/s]


plt_org_user


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  7.04it/s]


plt_intesrv_tycustomer


...

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 30.20it/s]


plt_intesrv_tylinkman


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 26.81it/s]


plt_intesrv_tytodorepair


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 28.32it/s]


In [71]:
graph = neo.getGraph()

In [112]:
def notNan(value):
    return value == value

def isNan(value):
    return value != value

In [72]:
a_old = graph.nodes.match("Person", name="Alice").first()

In [76]:
a_old

(_323961:Person {name: 'Alice'})

In [78]:
a = Node("Person", name="Alice")
b = Node("Person", name="Bob")
ab = Relationship(a, "KNOWS", b)
ab

(Alice)-[:KNOWS {}]->(Bob)

In [265]:
config.getConcept().reset_index().index

RangeIndex(start=0, stop=53, step=1)

In [68]:
ac

(Alice)-[:WORDS_WITH {}]->(Carol)

In [73]:
c = Node("Person", name="Carol")

ac = Relationship(a,'WORDS_WITH', c)
d = Node("Person", name="Yaoyao")

In [74]:
ad = Relationship(a,'knows',d)
ad

(Alice)-[:knows {}]->(Yaoyao)

In [234]:
transfer = ToolForTransferDataToNeo4j(config, entityInRdb, neo)

transfer.constructAllEntitiesAndRelations()

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

<class 'dict'>


AttributeError: 'dict' object has no attribute 'shape'

In [64]:
s = ab | ac | a | b | c
s

In [79]:
neo.createNode(a)

In [80]:
neo.createNode(b)

In [81]:
neo.createNode(ab)

In [65]:
neo.createNode(s)

In [75]:
neo.createNode(ad)

In [70]:
neo.createNode(ac|ab)

In [60]:
neo.createNode(ac|ab|a|b)

In [77]:
neo.deleteAllOrByLabel()

In [43]:
s.types()

frozenset({'KNOWS', 'WORDS_WITH'})

In [14]:
df_concept.head()

,concept_ch,display_name,table_and_column,type,inner_properties_ch,properties_col_str,code,compound_table_str,table,col,properties_col,properties_col_ch,compound_table
0,故障,故障名称,manual_fc.faultcode,直接,"故障名称,报警级别","faultname,alertlevel",nan,nan,manual_fc,faultcode,"[faultname, alertlevel]","[故障名称, 报警级别]",NaN
1,故障内容,unique_id,manual_fc.faultcontent,直接,nan,nan,nan,nan,manual_fc,faultcontent,[],[],NaN
2,机器发生的故障,unique_id,manual_fc.machinefault,直接,nan,nan,nan,nan,manual_fc,machinefault,[],[],NaN
3,控制器的反应,unique_id,manual_fc.controllerreaction,直接,nan,nan,nan,nan,manual_fc,controllerreaction,[],[],NaN
4,相关信息,unique_id,manual_fc.relatedinfo,直接,nan,nan,nan,nan,manual_fc,relatedinfo,[],[],NaN


In [25]:
nodes_concept = {}
for index in tqdm(list(df_concept.index)):
    index = int(index)
#     print(index)
#     print(df_concept.iloc[index]['concept_ch'])
    key = df_concept.iloc[index]['concept_ch']
    nodes_concept[key] = Node(key)
    nodes_concept[key].add_label("概念节点")
    if option_label != "":
        nodes_concept[key].add_label(option_label)
    # 为概念节点增加属性，把该有的东西都放进去
    for col in list(df_concept.columns.values):
        col_chinese = dict_concept_reverse[col] if col in dict_concept_reverse.keys() else col
        nodes_concept[key][col_chinese] = str(df_concept[col][index]) if df_concept[col][index] == df_concept[col][index] else ""
#     print(nodes_concept[key])
    graph.create(nodes_concept[key])

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 25.41it/s]


In [26]:
df_relation.head()

,id,sub,predicate,obj,table,type,prop_ch_str,prop_col_str,prop_ch,prop_col
0,1,故障,包含,故障内容,manual_fc,nan,nan,nan,[],[]
1,2,故障,包含,相关信息,manual_fc,nan,nan,nan,[],[]
2,3,故障,导致,机器发生的故障,manual_fc,nan,nan,nan,[],[]
3,4,故障,表现为,控制器的反应,manual_fc,nan,nan,nan,[],[]
4,5,负责系统检查的部件,检查,故障,manual_fc,nan,nan,nan,[],[]


In [27]:
for index in tqdm(list(df_relation.index)):
    index = int(index)
#     print(index)
    sub = df_relation.iloc[index]['sub']
    obj = df_relation.iloc[index]['obj']
#     try:
#     print(sub)
#     print(obj)
    aRelation = Relationship(nodes_concept[sub], df_relation.iloc[index]['predicate'], nodes_concept[obj] )
    aRelation["id_relation"] = df_relation.iloc[index]['id']
    for col in list(df_relation.columns.values):
        col_chinese = dict_relation_reverse[col] if col in dict_relation_reverse.keys() else col
        aRelation[col_chinese] = df_relation[col][index] if df_relation[col][index] == df_relation[col][index] else ""
     
    graph.create(aRelation)
#     except(KeyError):
#         pass

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  7.30it/s]


### 合成概念函数test

In [15]:
def test1(df):
#     print(df)
    return df['plt_lastmodifytime'][0] + " and " + df['plt_createtime'][0]

In [16]:
def test2(df):
#     print(df)
    return len(df)

In [17]:
def test3(df):
#     print(df)
    return len(df)

### 数据图

#### 单个表数据(debug)

#### 所有数据

In [18]:
def change2float(x):
    if re.match("^\d+?(\.\d+?)?$", str(x)) is None:
        return x
    else:
        return float(x)
change2float('1.09')

1.09

In [19]:
def transferOneRowToNeo4j():
    def establishNode():
        def establishDirectNode():
            pass
        def establishCompoundNode():
            pass
        establishDirectNode()
        establishCompoundNode()
        pass
    def establishRelation():
        def retrieveNode():
            pass
        def establishRelationWithNode():
            pass
        retrieveNode()
        establishRelationWithNode()
        pass
    establishNode()
    establishRelation()
    pass
transferOneRowToNeo4j()
# 考虑换成class，一个public函数，其余函数都是private

In [20]:
for table_name in tables_name:
    print(table_name)
    # 取出属于本表的几个小表，包括实体数据表、概念表、关系表
    df_table = tables[table_name]
    df_concept_one_table = df_concept[df_concept['table'] == table_name]
    df_relation_one_table = df_relation[df_relation['table'] == table_name]

    concept_ch_list_one_table = list(df_concept_one_table['concept_ch'])
    for i_row in tqdm(range(0,df_table.shape[0])): 
        transferOneRowToNeo4j()

manual_fc


100%|██████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<?, ?it/s]


In [28]:
# if option_deleteAllFlag == 1:
#     graph.delete_all()

# table_name = 'plt_intesrv_tydevice'
for table_name in tables_name:
    print(table_name)
    # 取出属于本表的几个小表，包括实体数据表、概念表、关系表
    df_table = tables[table_name]
    df_concept_one_table = df_concept[df_concept['table'] == table_name]
    df_relation_one_table = df_relation[df_relation['table'] == table_name]

    concept_ch_list_one_table = list(df_concept_one_table['concept_ch'])
    # 对于数据表的每行进行处理
    for i_row in tqdm(range(0,df_table.shape[0])): 
#         print(i_row)
        entity_node = {}
        # 对于概念表中的每个概念进行处理
        for i_entity in list(df_concept_one_table.index):
            key = df_concept_one_table['concept_ch'][i_entity] # eg:"客户","挖掘机"
            if df_concept_one_table['type'][i_entity] == '直接':
                value = df_table[ df_concept_one_table['col'][i_entity] ][i_row] # eg:"张三","PC200"
                if value == value: # 如果该节点的值非空
                    nodeBefore = graph.nodes.match(key, value=value).first()
                    if nodeBefore is None:
                        aNode = Node(key)
                        aNode['value'] = change2float(value)
                        aNode['type'] = key
                        aNode.add_label("实例节点") # Neo4j的官方label，支持高效索引

                        # 为该节点增加对应属性
                        assert len(df_concept_one_table['properties_col'][i_entity]) == len(df_concept_one_table['properties_col_ch'][i_entity]) \
                            , "properties_col=%s, properties_col_ch=%s. They have different length."%(str(df_concept_one_table['properties_col'][i_entity]), str(df_concept_one_table['properties_col_ch'][i_entity]))
                        prop_num = len(df_concept_one_table['properties_col'][i_entity])
                        for i_prop in range(0,prop_num):
                            prop_key = df_concept_one_table['properties_col'][i_entity][i_prop]  # eg: prop_key = 'plt_devicebrand'
                            prop_key_ch = df_concept_one_table['properties_col_ch'][i_entity][i_prop] # eg: prop_key_ch = '品牌'
                            prop_value = df_table[prop_key][i_row]
                            if prop_value == prop_value: 
                                aNode[prop_key_ch] = change2float(prop_value)
                            else: # 属性值如果为空的话，会置为空字符串
                                aNode[prop_key_ch] = ''

                        # 为该节点增加display_name属性
    #                     display = "(" + str(key) + ")"
                        display = str(key) + ":"
                        display_col = df_concept_one_table['display_name'][i_entity]
                        if display_col == 'unique_id':
                            display += value 
                        elif display_col == "nan": # 如果那一列为空的话，display就只显示概念名，不显示具体指
                            display_col = display_col[:-2]
                        else:
                            # 根据中文列名找到数据库列名
                            prop_num = len(df_concept_one_table['properties_col'][i_entity])
                            for i_prop in range(0,prop_num):
                                prop_key = df_concept_one_table['properties_col'][i_entity][i_prop]  # eg: prop_key = 'plt_devicebrand'
                                prop_key_ch = df_concept_one_table['properties_col_ch'][i_entity][i_prop] # eg: prop_key_ch = '品牌'
                                if prop_key_ch == display_col:
                                    break
                            assert i_prop != prop_num - 1 or (i_prop == prop_num - 1 and prop_key_ch == display_col) \
                                , "display_col=%s, properties_col_ch=%s. display_col doesn't exist in properties_col_ch."%(display_col, str(df_concept_one_table['properties_col_ch'][i_entity]))
                            display += str(df_table[prop_key][i_row])
    #                     print("debug:display=", display)
                        aNode['display_name'] = display
                        if option_label != "":
                            aNode.add_label(option_label)
    #                     print(aNode)
                        graph.create(aNode)
                    else:
                        aNode = nodeBefore
                    entity_node[key] = aNode
            # 生成合成概念实体
            elif df_concept_one_table['type'][i_entity] == '合成概念':
#                 print(df_concept_one_table['concept_ch'][i_entity])
#                 print(df_concept_one_table['compound_table'][i_entity])
                if str(df_concept_one_table['compound_table'][i_entity]) == 'nan': # 非空的话只需要把当前所在行传入函数
                    indexs = [] 
                    indexs.append(i_row)
                    # use list as index to return DataFrame instead of Series though the list has only one element
                    df_compound_need = df_table.loc[indexs,:].reset_index(drop=True) 
                else: # 处理联合多表的情况
                    compound = df_concept_one_table['compound_table'][i_entity][0]
                    assert table_name == compound['main_table'] # 第一个跳转表的主表应该得是本表
                    main_key_value = df_table[compound['main_col']][i_row]
                    df_table_new = tables[compound['foreign_table']]
                    foreign_col = compound['foreign_col']
                    df_compound_need = df_table_new.loc[df_table_new[foreign_col] == main_key_value]
                    # TODO:处理联合多表的情况
                value = eval(df_concept_one_table['code'][i_entity] + "(df_compound_need)")
                aNode = Node(key)
                aNode['value'] = change2float(value)
                aNode['type'] = key
                aNode['display_name'] = str(key) + ":" + str(value) 
                if option_label != "":
                    aNode.add_label(option_label)
                graph.create(aNode)
                entity_node[key] = aNode
    #     print(entity_node)
    
        # 建立本行数据中的节点间的关系
        for i_relation in list(df_relation_one_table.index):
    #         try: 
            # 建立sub主语实体，有可能已经在entity_node中，也有可能是外键
            entity_ch = df_relation_one_table['sub'][i_relation]
            if entity_ch in entity_node.keys(): # 本行数据确实有该节点
                entity = entity_node[entity_ch]
            elif entity_ch not in concept_ch_list_one_table: # 本行数据无该节点，则为外键
                # entity_ch(eg:客户)去模式表里找该概念的数据库主键来源，根据主键和当前表table_name来找到该概念在当前表的外键列名，然后根据该外键列名找到外键具体值，去neo4j里查询该节点，
#                 print(entity_ch)
                primary_key_table = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'table'].item()
                primary_key_col = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'col'].item()
#                 print(primary_key_table)
#                 print(primary_key_col)
#                 print(entity_ch)
#                 print(df_foreign['primary_table'] == primary_key_table)
#                 print((df_foreign['primary_key'] == primary_key_col))
#                 print((df_foreign['foreign_table'] == table_name))
                foreign_key = df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
                                  & (df_foreign['primary_key'] == primary_key_col)
                                  & (df_foreign['foreign_table'] == table_name), 'foreign_key'].item()
#                 print(foreign_key)
                key = entity_ch
                value = df_table[foreign_key][i_row]
                if value != value: # 虽然该键是外键，但是该键依然可能是控制(玄学数据)
                    continue
                nodeBefore = graph.nodes.match(key, value=value).first()
    #             assert nodeBefore is not None
                if nodeBefore is None: # 这是测试数据的锅，否则的话永远不应该为None的
                    continue
                entity = nodeBefore
    #             print(type(primary_key_table)) 
    #             primary_key_table, primary_key_col
    #             continue
            else: # 本行数据的该节点value为空，所以没有该节点
                continue
            sub = entity

            # 建立obj宾语实体，这二三十航代码都是抄的上面的，只要把首末行的sub改成obj就可以了
            entity_ch = df_relation_one_table['obj'][i_relation]
            if entity_ch in entity_node.keys(): # 本行数据确实有该节点
                entity = entity_node[entity_ch]
            elif entity_ch not in concept_ch_list_one_table: # 本行数据无该节点，则为外键
                # 根据entity_ch(eg:客户)去模式表里找该概念的数据库主键来源，根据主键和当前表table_name来找到该概念在当前表的外键列名，然后根据该外键列名找到外键具体值，去neo4j里查询该节点，
#                 print(entity_ch)
                primary_key_table = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'table'].item()
                primary_key_col = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'col'].item()
#                 print(primary_key_table)
#                 print(primary_key_col)
#                 print(entity_ch)
#                 print(df_foreign['primary_table'] == primary_key_table)
#                 print((df_foreign['primary_key'] == primary_key_col))
#                 print((df_foreign['foreign_table'] == table_name))
#                 print("-"*10)
#                 print(df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
#                                   & (df_foreign['primary_key'] == primary_key_col)
#                                   & (df_foreign['foreign_table'] == table_name), 'foreign_key'])
#                 print("-"*10)
                foreign_key = df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
                                  & (df_foreign['primary_key'] == primary_key_col)
                                  & (df_foreign['foreign_table'] == table_name), 'foreign_key'].item()
#                 print(foreign_key)
                key = entity_ch
                value = df_table[foreign_key][i_row]
                if value != value: # 虽然该键是外键，但是该键依然可能是空值(玄学数据)
                    continue
                nodeBefore = graph.nodes.match(key, value=value).first()
    #             assert nodeBefore is not None
                if nodeBefore is None: # 测试数据的锅，否则的话永远不应该为None的
                    continue            
                entity = nodeBefore
    #             print(type(primary_key_table))
    #             primary_key_table, primary_key_col
    #             continue
            else: # 本行数据的该节点value为空，所以没有该节点
                continue
            obj = entity

            # 建立主宾实体间的关系
            predicate = df_relation_one_table['predicate'][i_relation]
            aRelation = Relationship(sub, predicate, obj)
            aRelation['id_relation'] = df_relation_one_table['id'][i_relation]
            # 为关系增加属性
            assert len(df_relation_one_table['prop_ch'][i_relation]) == len(df_relation_one_table['prop_col'][i_relation]) \
                , "prop_ch=%s, prop_col=%s. They have different length."%(str(df_relation_one_table['prop_ch'][i_relation]), str(df_relation_one_table['prop_col'][i_relation]))
            prop_num = len(df_relation_one_table['prop_ch'][i_relation])
            for i_prop in range(0,prop_num):
                prop_key = df_relation_one_table['prop_col'][i_relation][i_prop]
                prop_key_ch = df_relation_one_table['prop_ch'][i_relation][i_prop]
                prop_value = df_table[prop_key][i_row]
                if prop_value == prop_value:
                    aRelation[prop_key_ch] = change2float(prop_value)
                else:
                    aRelation[prop_key_ch] = ''

            graph.create(aRelation)
    #         except KeyError: # 因为某些节点可能因为值为空所以没有该节点，所以在entity_node字典中没有该节点
    #             pass
    #         print(entity_name)

manual_fc


100%|████████████████████████████████████████████████████████████████████████████████| 538/538 [00:34<00:00, 15.53it/s]
